In [86]:
import numpy as np
import pandas as pd
import scipy
from scipy.optimize import minimize
import plotly.express as px
import plotly.graph_objects as go
import datetime

from sklearn import linear_model as lm
from sklearn.feature_selection import r_regression
from sklearn.metrics import r2_score

In [87]:
import warnings
warnings.filterwarnings('ignore')

In [88]:
approval = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vS-FKWVTTFtJT6u56e0bqdfoMcXvDO1DUChsJ3jQAMB2lZk2SMqVfmg7dGjclTYkYWz-Pm5lfcLPjp4/pub?output=csv')
# approval = approval.reset_index()
# approval = approval.rename({'index':'poll_id'}, axis=1)
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,tracking,approve,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp
0,Donald Trump (2nd term),Cost,Beacon Research/Shaw & Co. Research,FOX News,6/13/2025,6/16/2025,1003.0,RV,1.418927,1.418927,NaN,34.0,64.0,-30.0,34.51359,63.199100,-28.685510,https://static.foxnews.com/foxnews.com/content...,6/16/2025,17:00:44 21 Jun 2025
1,Donald Trump (2nd term),Adults,Ipsos,Reuters,6/11/2025,6/16/2025,4258.0,A,1.354591,1.277143,NaN,42.0,54.0,-12.0,43.02135,53.866900,-10.845550,https://www.reuters.com/world/us/trump-approva...,6/16/2025,17:00:35 21 Jun 2025
2,Donald Trump (2nd term),Strong,Echelon Insights,NaN,6/17/2025,6/18/2025,1002.0,LV,1.258036,1.258036,NaN,26.0,41.0,-15.0,26.29254,41.349300,-15.056760,https://echelonin.wpenginepowered.com/wp-conte...,6/18/2025,17:00:37 21 Jun 2025
3,Donald Trump (2nd term),Strong,Beacon Research/Shaw & Co. Research,FOX News,6/13/2025,6/16/2025,1003.0,RV,1.418927,1.176594,NaN,27.0,43.0,-16.0,27.28083,42.277400,-14.996570,https://static.foxnews.com/foxnews.com/content...,6/16/2025,17:00:37 21 Jun 2025
4,Donald Trump (2nd term),Economy,Beacon Research/Shaw & Co. Research,FOX News,6/13/2025,6/16/2025,1003.0,RV,1.418927,1.176594,NaN,40.0,58.0,-18.0,40.95854,56.720160,-15.761620,https://static.foxnews.com/foxnews.com/content...,6/16/2025,17:00:41 21 Jun 2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1608,Donald Trump (2nd term),Weak,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.000000,NaN,15.0,8.0,7.0,13.06756,7.217384,5.850176,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,17:00:39 21 Jun 2025
1609,Donald Trump (2nd term),Trade,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.996709,0.000000,NaN,38.0,40.0,-2.0,40.19163,40.207080,-0.015449,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,17:00:45 21 Jun 2025
1610,Donald Trump (2nd term),Strong,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.000000,NaN,29.0,29.0,0.0,33.26019,29.380020,3.880169,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,17:00:37 21 Jun 2025
1611,Donald Trump (2nd term),Voters,InsiderAdvantage,NaN,1/20/2025,1/20/2025,800.0,RV,1.093180,0.000000,NaN,56.0,39.0,17.0,53.62980,41.019580,12.610220,https://insideradvantage.com/first-insideradva...,1/20/2025,17:00:32 21 Jun 2025


In [89]:
approval = approval[~approval['subgroup'].isin(['Weak', 'Strong', 'Economy', 'Cost', 'Immigration', 'Trade', 'Adults', 'Voters'])]

In [90]:
approval.to_csv('trump_approval_polls.csv')

In [91]:
approval.columns.values

array(['president', 'subgroup', 'pollster', 'sponsors', 'startdate',
       'enddate', 'samplesize', 'population', 'weight', 'influence',
       'tracking', 'approve', 'disapprove', 'net', 'adjusted_approve',
       'adjusted_disapprove', 'adjusted_net', 'url', 'createddate',
       'timestamp'], dtype=object)

In [92]:
approval.shape

(350, 20)

In [93]:
# dates = approval['Dates'].str.extract(r'(\d{1,2}\/\d{1,2}) \- (\d{1,2}\/\d{1,2}), (\d{4})')
# dates

In [94]:
# dates['end_date'] = dates[1] + '/' + dates[2]
# dates['end_date'] = pd.to_datetime(dates['end_date'])
# dates

In [95]:
# approval['Dates'] = dates['end_date']
# approval.head()

In [96]:
# samples = approval['Sample'].str.extract(r'[0-9,]{3,5} (\w{1,2})@@(\d{3,4})')
# samples

In [97]:
# approval['population'] = samples[0]
# approval['sample_size'] = samples[1].astype(int)
# approval.head()

In [98]:
ratings = pd.read_csv('pollster_ratings_silver.csv')
ratings

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
0,AtlasIntel,A+@@1,-0.957740,D+0.16@@0.16334188,91,D+
1,Marquette University Law School,A/B@@4,-0.920261,D+0.32@@0.31545374,17,D+
2,The Washington Post,A+@@1,-0.902281,D+0.90@@0.89811021,98,D+
3,Research & Polling Inc.,A@@2,-0.871136,R+0.55@@-0.54944503,44,R+
4,The New York Times/Siena College,A@@2,-0.763947,D+1.41@@1.4121077,116,D+
...,...,...,...,...,...,...
530,Nielson Brothers Polling,C/D@@12,1.946864,D+3.05@@3.0515239,11,D+
531,Ascend Action,C/D@@12,2.282717,D+3.35@@3.3485439,6,D+
532,Jayhawk Consulting Services,C/D@@12,2.353968,D+2.27@@2.2674108,2,D+
533,Fort Hays State University,C/D@@12,2.489170,R+1.10@@-1.0960908,6,R+


In [99]:
ratings['Pollster'].values

array(['AtlasIntel', 'Marquette University Law School',
       'The Washington Post', 'Research & Polling Inc.',
       'The New York Times/Siena College', 'Cygnal', 'SurveyUSA',
       'Siena College', 'Beacon Research/Shaw & Co. Research',
       'Alaska Survey Research', 'OnMessage Inc.', 'TIPP Insights',
       'Emerson College', 'Landmark Communications',
       'Monmouth University', 'Data Orbital',
       'University of North Florida', 'MassINC Polling Group',
       'Marist College', 'Public Policy Institute of California',
       'Public Policy Polling', 'University of California Berkeley',
       'CBS News/The New York Times', 'Christopher Newport University',
       'Mason-Dixon Polling & Strategy', 'Strategic Vision LLC',
       'Suffolk University', 'Stockton University',
       'Field Research Corp. (Field Poll)', 'Quinnipiac University',
       'Los Angeles Times',
       'Hart Research Associates/Public Opinion Strategies',
       'Public Opinion Strategies', 'Susquehan

In [100]:
ratings[ratings['Pollster'].str.contains('North Star')]#.sum()

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
75,North Star Opinion Research,B/C@@8,0.043339,R+0.63@@-0.62697512,13,R+
425,Univision/Latino Decisions/North Star Opinion ...,B/C@@8,0.830409,D+1.06@@1.0564657,7,D+


In [101]:
banned_pollsters = ratings[ratings['Grade'].str.contains('F')]['Pollster'].values
banned_pollsters

array(['Strategic Vision LLC', 'Pharos Research Group', 'Research 2000',
       'Big Data Poll', 'Overtime Politics', 'Rethink Priorities',
       'Blumenthal Research Daily', 'CSP Polling', 'KG Polling',
       'OurProgress (The Progress Campaign)', 'TCJ Research'],
      dtype=object)

In [102]:
approval = approval[~approval['pollster'].isin(banned_pollsters)]
approval.shape

(350, 20)

In [103]:
approval = pd.merge(left=approval, right=ratings[['Pollster', 'Predictive Plus-Minus']], left_on='pollster',
                   right_on='Pollster', how='inner')
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus
0,Donald Trump (2nd term),All polls,Beacon Research/Shaw & Co. Research,FOX News,6/13/2025,6/16/2025,1003.0,RV,1.418927,1.176594,...,54.0,-8.000000,45.78914,53.26493,-7.475792,https://static.foxnews.com/foxnews.com/content...,6/16/2025,17:00:29 21 Jun 2025,Beacon Research/Shaw & Co. Research,-0.635562
1,Donald Trump (2nd term),All polls,Ipsos,Reuters,6/11/2025,6/16/2025,4258.0,A,1.354591,1.044800,...,54.0,-12.000000,44.04047,52.98749,-8.947021,https://www.reuters.com/world/us/trump-approva...,6/16/2025,17:00:29 21 Jun 2025,Ipsos,0.442898
2,Donald Trump (2nd term),All polls,Echelon Insights,NaN,6/17/2025,6/18/2025,1002.0,LV,1.027769,1.027769,...,52.0,-4.000000,47.54914,52.05284,-4.503700,https://echelonin.wpenginepowered.com/wp-conte...,6/18/2025,17:00:29 21 Jun 2025,Echelon Insights,-0.216672
3,Donald Trump (2nd term),All polls,InsiderAdvantage,NaN,6/15/2025,6/16/2025,1000.0,LV,1.153754,1.021995,...,44.1,10.300000,51.62939,46.49978,5.129612,https://insideradvantage.com/insideradvantage-...,6/16/2025,17:00:29 21 Jun 2025,InsiderAdvantage,0.034562
4,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,6/11/2025,6/12/2025,2097.0,RV,1.318091,0.865357,...,50.0,-4.000000,44.81514,51.83273,-7.017590,https://harvardharrispoll.com/wp-content/uploa...,6/12/2025,17:00:29 21 Jun 2025,Harris Insights & Analytics,0.162447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,1/20/2025,1/24/2025,1500.0,LV,0.213379,0.000000,...,43.0,9.000000,49.10089,44.62543,4.475460,https://www.rasmussenreports.com/public_conten...,1/24/2025,17:00:29 21 Jun 2025,Rasmussen Reports,0.087015
330,Donald Trump (2nd term),All polls,RMG Research,Napolitan News Service,1/20/2025,1/23/2025,3000.0,RV,0.927446,0.000000,...,39.0,18.000000,53.72900,41.39184,12.337160,https://x.com/ScottWRasmussen/status/188280455...,1/23/2025,17:00:29 21 Jun 2025,RMG Research,0.278023
331,Donald Trump (2nd term),All polls,AtlasIntel,NaN,1/21/2025,1/23/2025,1882.0,A,1.910964,0.000000,...,49.7,0.399998,49.89090,48.36575,1.525150,https://cdn.atlasintel.org/ed3f5238-e823-495e-...,1/23/2025,17:00:29 21 Jun 2025,AtlasIntel,-0.957740
332,Donald Trump (2nd term),All polls,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.000000,...,41.0,6.000000,49.04047,39.98749,9.052979,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,17:00:29 21 Jun 2025,Ipsos,0.442898


In [104]:
approval.shape

(334, 22)

In [105]:
approval['Pollster'].value_counts()

Pollster
Rasmussen Reports                                     106
YouGov                                                 40
Morning Consult                                        26
RMG Research                                           22
Ipsos                                                  15
J.L. Partners                                          11
Echelon Insights                                        8
TIPP Insights                                           8
Harris Insights & Analytics                             7
Global Strategy Group/GBAO (Navigator Research)         7
Gallup                                                  5
AtlasIntel                                              5
Quinnipiac University                                   5
Emerson College                                         5
Clarity Campaign Labs                                   4
SSRS                                                    4
McLaughlin & Associates                                 4
Ameri

In [106]:
# approval[~approval['tracking'].isna()]

In [107]:
approval['tracking'].value_counts()

tracking
True    110
Name: count, dtype: int64

In [108]:
# full_approval = approval.copy()

In [109]:
approval = approval[(~approval['samplesize'].isna())]
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus
0,Donald Trump (2nd term),All polls,Beacon Research/Shaw & Co. Research,FOX News,6/13/2025,6/16/2025,1003.0,RV,1.418927,1.176594,...,54.0,-8.000000,45.78914,53.26493,-7.475792,https://static.foxnews.com/foxnews.com/content...,6/16/2025,17:00:29 21 Jun 2025,Beacon Research/Shaw & Co. Research,-0.635562
1,Donald Trump (2nd term),All polls,Ipsos,Reuters,6/11/2025,6/16/2025,4258.0,A,1.354591,1.044800,...,54.0,-12.000000,44.04047,52.98749,-8.947021,https://www.reuters.com/world/us/trump-approva...,6/16/2025,17:00:29 21 Jun 2025,Ipsos,0.442898
2,Donald Trump (2nd term),All polls,Echelon Insights,NaN,6/17/2025,6/18/2025,1002.0,LV,1.027769,1.027769,...,52.0,-4.000000,47.54914,52.05284,-4.503700,https://echelonin.wpenginepowered.com/wp-conte...,6/18/2025,17:00:29 21 Jun 2025,Echelon Insights,-0.216672
3,Donald Trump (2nd term),All polls,InsiderAdvantage,NaN,6/15/2025,6/16/2025,1000.0,LV,1.153754,1.021995,...,44.1,10.300000,51.62939,46.49978,5.129612,https://insideradvantage.com/insideradvantage-...,6/16/2025,17:00:29 21 Jun 2025,InsiderAdvantage,0.034562
4,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,6/11/2025,6/12/2025,2097.0,RV,1.318091,0.865357,...,50.0,-4.000000,44.81514,51.83273,-7.017590,https://harvardharrispoll.com/wp-content/uploa...,6/12/2025,17:00:29 21 Jun 2025,Harris Insights & Analytics,0.162447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,1/20/2025,1/24/2025,1500.0,LV,0.213379,0.000000,...,43.0,9.000000,49.10089,44.62543,4.475460,https://www.rasmussenreports.com/public_conten...,1/24/2025,17:00:29 21 Jun 2025,Rasmussen Reports,0.087015
330,Donald Trump (2nd term),All polls,RMG Research,Napolitan News Service,1/20/2025,1/23/2025,3000.0,RV,0.927446,0.000000,...,39.0,18.000000,53.72900,41.39184,12.337160,https://x.com/ScottWRasmussen/status/188280455...,1/23/2025,17:00:29 21 Jun 2025,RMG Research,0.278023
331,Donald Trump (2nd term),All polls,AtlasIntel,NaN,1/21/2025,1/23/2025,1882.0,A,1.910964,0.000000,...,49.7,0.399998,49.89090,48.36575,1.525150,https://cdn.atlasintel.org/ed3f5238-e823-495e-...,1/23/2025,17:00:29 21 Jun 2025,AtlasIntel,-0.957740
332,Donald Trump (2nd term),All polls,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.000000,...,41.0,6.000000,49.04047,39.98749,9.052979,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,17:00:29 21 Jun 2025,Ipsos,0.442898


In [110]:
auxiliary = pd.read_csv('auxiliary_pollster_rating_stats.csv')
auxiliary

,Pollster,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat
0,AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+
1,The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+
2,Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+
3,The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+
4,Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+
...,...,...,...,...,...,...,...,...
93,Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+
94,Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+
95,Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+
96,McLaughlin & Associates,0.689655,34.5%@@65.517241,R+4.94@@-4.9392128,6.2@@6.17804,0.000000,1,R+


In [111]:
house_eff = auxiliary['House effect'].str.extract(r'(\w)\+(\d\.\d\d)@@*.')
house_eff[1] = house_eff[1].astype(float)
house_eff

,0,1
0,R,2.00
1,D,1.02
2,D,0.63
3,D,0.79
4,R,0.60
...,...,...
93,R,2.44
94,R,0.45
95,R,3.23
96,R,4.94


In [112]:
# R --> -1, D --> 1
house_eff[0] = house_eff[0].replace({'R':-1, 'D':1})
house_eff['house_effect'] = house_eff[0] * house_eff[1]
house_eff

,0,1,house_effect
0,-1,2.00,-2.00
1,1,1.02,1.02
2,1,0.63,0.63
3,1,0.79,0.79
4,-1,0.60,-0.60
...,...,...,...
93,-1,2.44,-2.44
94,-1,0.45,-0.45
95,-1,3.23,-3.23
96,-1,4.94,-4.94


In [113]:
auxiliary['house_effect'] = house_eff['house_effect']
auxiliary = auxiliary.set_index(['Pollster'])
auxiliary

,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat,house_effect
Pollster,,,,,,,,
AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+,-2.00
The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+,1.02
Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+,0.63
The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+,0.79
Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+,-0.60
...,...,...,...,...,...,...,...,...
Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+,-2.44
Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+,-0.45
Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+,-3.23


In [114]:
def apply_house_eff(data, pollster):
    df = data[data['pollster']==pollster]
    try:
        df['approve_adj'] = df['approve'] + (auxiliary.loc[pollster, 'house_effect'] / 2)
        df['disapprove_adj'] = df['disapprove'] - (auxiliary.loc[pollster, 'house_effect'] / 2)
    except KeyError:
        df['approve_adj'] = df['approve']
        df['disapprove_adj'] = df['disapprove']
    return df

In [115]:
pollsters = approval['pollster'].value_counts().index.values
df_adjs = []
for p in pollsters:
    df = apply_house_eff(approval, p)
    df_adjs.append(df)

approval_adj = pd.concat(df_adjs, axis=0)
approval_adj

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus,approve_adj,disapprove_adj
8,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/13/2025,6/19/2025,1500.0,LV,0.842307,0.740202,...,49.10089,48.62543,0.475460,https://www.rasmussenreports.com/public_conten...,6/19/2025,17:00:29 21 Jun 2025,Rasmussen Reports,0.087015,51.360,47.640
26,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/6/2025,6/12/2025,1500.0,LV,0.324773,0.144533,...,50.10089,46.62543,3.475460,https://www.rasmussenreports.com/public_conten...,6/12/2025,17:00:29 21 Jun 2025,Rasmussen Reports,0.087015,52.360,45.640
31,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/12/2025,6/18/2025,1500.0,LV,0.117946,0.096451,...,49.10089,48.62543,0.475460,https://www.rasmussenreports.com/public_conten...,6/18/2025,17:00:29 21 Jun 2025,Rasmussen Reports,0.087015,51.360,47.640
32,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/9/2025,6/13/2025,1500.0,LV,0.155384,0.088643,...,49.10089,47.62543,1.475460,https://www.rasmussenreports.com/public_conten...,6/13/2025,17:00:29 21 Jun 2025,Rasmussen Reports,0.087015,51.360,46.640
33,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/11/2025,6/17/2025,1500.0,LV,0.115884,0.087662,...,49.10089,48.62543,0.475460,https://www.rasmussenreports.com/public_conten...,6/17/2025,17:00:29 21 Jun 2025,Rasmussen Reports,0.087015,51.360,47.640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,Donald Trump (2nd term),All polls,North Star Opinion Research,Defending Democracy Together,3/13/2025,3/17/2025,1000.0,RV,1.158492,0.000000,...,46.28450,50.75379,-4.469292,https://www.defendingdemocracytogether.org/app...,3/17/2025,17:00:29 21 Jun 2025,North Star Opinion Research,0.043339,46.000,51.000
98,Donald Trump (2nd term),All polls,University of Massachusetts (Amherst),WCVB-TV,4/4/2025,4/9/2025,1000.0,A,0.983206,0.000000,...,44.30865,51.05889,-6.750240,https://www.umass.edu/political-science/about/...,4/9/2025,17:00:29 21 Jun 2025,University of Massachusetts (Amherst),0.536200,44.000,51.000
283,Donald Trump (2nd term),All polls,OnMessage Inc.,Rescuing the American Dream,2/17/2025,2/19/2025,1000.0,LV,1.404286,0.000000,...,49.51034,48.58600,0.924343,https://onmessageinc.com/wp-content/uploads/20...,2/19/2025,17:00:29 21 Jun 2025,OnMessage Inc.,-0.600249,48.225,50.775
298,Donald Trump (2nd term),All polls,SurveyUSA,NaN,2/13/2025,2/16/2025,2000.0,A,1.824873,0.000000,...,50.24733,45.72145,4.525879,https://www.surveyusa.com/client/PollReport.as...,2/16/2025,17:00:29 21 Jun 2025,SurveyUSA,-0.732433,50.765,45.235


In [116]:
approval.shape

(334, 22)

In [117]:
approval = approval_adj.copy()
approval.shape

(334, 24)

In [118]:
approval['poll_id'] = pd.factorize(approval['pollster'] + approval['enddate'])[0]
approval.head(7)

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus,approve_adj,disapprove_adj,poll_id
8,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/13/2025,6/19/2025,1500.0,LV,0.842307,0.740202,...,48.62543,0.47546,https://www.rasmussenreports.com/public_conten...,6/19/2025,17:00:29 21 Jun 2025,Rasmussen Reports,0.087015,51.36,47.64,0
26,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/6/2025,6/12/2025,1500.0,LV,0.324773,0.144533,...,46.62543,3.47546,https://www.rasmussenreports.com/public_conten...,6/12/2025,17:00:29 21 Jun 2025,Rasmussen Reports,0.087015,52.36,45.64,1
31,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/12/2025,6/18/2025,1500.0,LV,0.117946,0.096451,...,48.62543,0.47546,https://www.rasmussenreports.com/public_conten...,6/18/2025,17:00:29 21 Jun 2025,Rasmussen Reports,0.087015,51.36,47.64,2
32,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/9/2025,6/13/2025,1500.0,LV,0.155384,0.088643,...,47.62543,1.47546,https://www.rasmussenreports.com/public_conten...,6/13/2025,17:00:29 21 Jun 2025,Rasmussen Reports,0.087015,51.36,46.64,3
33,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/11/2025,6/17/2025,1500.0,LV,0.115884,0.087662,...,48.62543,0.47546,https://www.rasmussenreports.com/public_conten...,6/17/2025,17:00:29 21 Jun 2025,Rasmussen Reports,0.087015,51.36,47.64,4
34,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/10/2025,6/16/2025,1500.0,LV,0.114323,0.079398,...,48.62543,0.47546,https://www.rasmussenreports.com/public_conten...,6/16/2025,17:00:29 21 Jun 2025,Rasmussen Reports,0.087015,51.36,47.64,5
36,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,5/30/2025,6/5/2025,1500.0,LV,0.319266,0.061393,...,50.62543,-3.52454,https://www.rasmussenreports.com/public_conten...,6/5/2025,17:00:29 21 Jun 2025,Rasmussen Reports,0.087015,49.36,49.64,6


In [119]:
approval['population'] = approval['population'].str.strip(' ')

In [120]:
def pipeline(data: pd.DataFrame):
    dup = data[data.duplicated(subset='poll_id', keep=False)]
    ndup = data[~data.duplicated(subset='poll_id', keep=False)]
    # print('dup', dup, '\n ndup', ndup)
    dup_a = dup[dup['population'].isin(['A'])]
    # print('dup_a', dup_a)
    df = pd.concat([dup_a, ndup], axis=0)
    
    dup = df[df.duplicated(subset='poll_id', keep=False)]
    ndup = df[~df.duplicated(subset='poll_id', keep=False)]
    dup_lv = dup[dup['population'].isin(['RV'])]
    # print('dup_rv', dup_lv, 'ndup', ndup)
    df = pd.concat([dup_lv, ndup], axis=0)
    
    df['end_date'] = pd.to_datetime(df['end_date'])
    
    return df

In [121]:
approval = approval.rename({'enddate':'end_date', 'samplesize':'sample_size', 'Predictive Plus-Minus':'pred_plus_minus'}, axis=1)

In [122]:
approval['population'].value_counts().index.values

array(['LV', 'RV', 'A', 'V'], dtype=object)

In [123]:
approval.shape, pipeline(approval).shape

((334, 25), (324, 25))

In [124]:
pipeline(approval)

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id
8,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/13/2025,2025-06-19,1500.0,LV,0.842307,0.740202,...,48.62543,0.475460,https://www.rasmussenreports.com/public_conten...,6/19/2025,17:00:29 21 Jun 2025,Rasmussen Reports,0.087015,51.360,47.640,0
26,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/6/2025,2025-06-12,1500.0,LV,0.324773,0.144533,...,46.62543,3.475460,https://www.rasmussenreports.com/public_conten...,6/12/2025,17:00:29 21 Jun 2025,Rasmussen Reports,0.087015,52.360,45.640,1
31,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/12/2025,2025-06-18,1500.0,LV,0.117946,0.096451,...,48.62543,0.475460,https://www.rasmussenreports.com/public_conten...,6/18/2025,17:00:29 21 Jun 2025,Rasmussen Reports,0.087015,51.360,47.640,2
32,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/9/2025,2025-06-13,1500.0,LV,0.155384,0.088643,...,47.62543,1.475460,https://www.rasmussenreports.com/public_conten...,6/13/2025,17:00:29 21 Jun 2025,Rasmussen Reports,0.087015,51.360,46.640,3
33,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/11/2025,2025-06-17,1500.0,LV,0.115884,0.087662,...,48.62543,0.475460,https://www.rasmussenreports.com/public_conten...,6/17/2025,17:00:29 21 Jun 2025,Rasmussen Reports,0.087015,51.360,47.640,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,Donald Trump (2nd term),All polls,North Star Opinion Research,Defending Democracy Together,3/13/2025,2025-03-17,1000.0,RV,1.158492,0.000000,...,50.75379,-4.469292,https://www.defendingdemocracytogether.org/app...,3/17/2025,17:00:29 21 Jun 2025,North Star Opinion Research,0.043339,46.000,51.000,324
98,Donald Trump (2nd term),All polls,University of Massachusetts (Amherst),WCVB-TV,4/4/2025,2025-04-09,1000.0,A,0.983206,0.000000,...,51.05889,-6.750240,https://www.umass.edu/political-science/about/...,4/9/2025,17:00:29 21 Jun 2025,University of Massachusetts (Amherst),0.536200,44.000,51.000,325
283,Donald Trump (2nd term),All polls,OnMessage Inc.,Rescuing the American Dream,2/17/2025,2025-02-19,1000.0,LV,1.404286,0.000000,...,48.58600,0.924343,https://onmessageinc.com/wp-content/uploads/20...,2/19/2025,17:00:29 21 Jun 2025,OnMessage Inc.,-0.600249,48.225,50.775,326
298,Donald Trump (2nd term),All polls,SurveyUSA,NaN,2/13/2025,2025-02-16,2000.0,A,1.824873,0.000000,...,45.72145,4.525879,https://www.surveyusa.com/client/PollReport.as...,2/16/2025,17:00:29 21 Jun 2025,SurveyUSA,-0.732433,50.765,45.235,327


In [125]:
# approval[approval['pollster']=='Harris Insights & Analytics']

In [126]:
full_approval = approval.copy()

In [127]:
approval = approval[approval['tracking'].isna()]

In [128]:
# banned_pollsters_aux = ['Rasmussen Reports', 'co/efficient', 'TIPP Insights', 'Civiqs', 'Trafalgar Group', 'OnMessage Inc.']
banned_pollsters_aux = pd.read_csv('banned_pollsters_aux.csv')['banned_pollsters'].values
banned_pollsters_aux

array(['Rasmussen Reports', 'TIPP Insights', 'Trafalgar Group',
       'McLaughlin & Associates', 'Trafalgar Group/Insider Advantage'],
      dtype=object)

In [129]:
approval = approval[~approval['pollster'].isin(banned_pollsters_aux)]
approval.head()

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id
10,Donald Trump (2nd term),All polls,YouGov,Economist,6/13/2025,6/16/2025,1512.0,A,0.839664,0.681783,...,54.42362,-12.56728,https://d3nkl3psvxxpe9.cloudfront.net/document...,6/16/2025,17:00:29 21 Jun 2025,YouGov,0.047635,41.25,53.75,105
21,Donald Trump (2nd term),All polls,YouGov,Economist,6/6/2025,6/9/2025,1533.0,A,0.636911,0.241767,...,52.42362,-8.56728,https://d3nkl3psvxxpe9.cloudfront.net/document...,6/9/2025,17:00:29 21 Jun 2025,YouGov,0.047635,43.25,51.75,106
23,Donald Trump (2nd term),All polls,YouGov,CBS News,6/4/2025,6/6/2025,2428.0,A,0.727909,0.191716,...,55.42362,-9.56728,https://www.cbsnews.com/news/deportation-immig...,6/6/2025,17:00:29 21 Jun 2025,YouGov,0.047635,45.25,54.75,107
30,Donald Trump (2nd term),All polls,YouGov,Economist,5/30/2025,6/2/2025,1610.0,A,0.583608,0.099467,...,49.42362,-3.56728,https://d3nkl3psvxxpe9.cloudfront.net/document...,6/2/2025,17:00:29 21 Jun 2025,YouGov,0.047635,45.25,48.75,108
42,Donald Trump (2nd term),All polls,YouGov,Economist,5/23/2025,5/26/2025,1660.0,A,0.565511,0.029300,...,52.42362,-7.56728,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/26/2025,17:00:29 21 Jun 2025,YouGov,0.047635,44.25,51.75,109


In [130]:
approval[approval['pollster']=='Civiqs']# [['approve', 'disapprove', 'approve_adj', 'disapprove_adj']]

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id
49,Donald Trump (2nd term),All polls,Civiqs,Daily Kos,5/17/2025,5/20/2025,1018.0,RV,1.120966,0.012053,...,51.26817,-4.653641,https://civiqs.com/documents/Civiqs_DailyKos_b...,5/20/2025,17:00:29 21 Jun 2025,Civiqs,0.158562,47.86,51.14,293
154,Donald Trump (2nd term),All polls,Civiqs,Daily Kos,4/12/2025,4/15/2025,1124.0,RV,1.149724,0.000000,...,52.26817,-6.653641,https://civiqs.com/reports/2025/04/17/report-a...,4/15/2025,17:00:29 21 Jun 2025,Civiqs,0.158562,46.86,52.14,294
193,Donald Trump (2nd term),All polls,Civiqs,Daily Kos,2/28/2025,3/3/2025,1031.0,RV,1.124632,0.000000,...,51.26817,-3.653641,https://civiqs.com/reports/2025/03/04/report-a...,3/3/2025,17:00:29 21 Jun 2025,Civiqs,0.158562,48.86,51.14,295


In [131]:
approval.columns.values

array(['president', 'subgroup', 'pollster', 'sponsors', 'startdate',
       'end_date', 'sample_size', 'population', 'weight', 'influence',
       'tracking', 'approve', 'disapprove', 'net', 'adjusted_approve',
       'adjusted_disapprove', 'adjusted_net', 'url', 'createddate',
       'timestamp', 'Pollster', 'pred_plus_minus', 'approve_adj',
       'disapprove_adj', 'poll_id'], dtype=object)

In [132]:
approval_polls = approval[['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size', 'population', 'approve', 'disapprove', 'approve_adj', 'disapprove_adj', 'net', 'url', 'pred_plus_minus', 'poll_id']]
approval_polls

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id
10,YouGov,Economist,6/13/2025,6/16/2025,1512.0,A,41.0,54.0,41.250,53.750,-13.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,105
21,YouGov,Economist,6/6/2025,6/9/2025,1533.0,A,43.0,52.0,43.250,51.750,-9.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,106
23,YouGov,CBS News,6/4/2025,6/6/2025,2428.0,A,45.0,55.0,45.250,54.750,-10.0,https://www.cbsnews.com/news/deportation-immig...,0.047635,107
30,YouGov,Economist,5/30/2025,6/2/2025,1610.0,A,45.0,49.0,45.250,48.750,-4.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,108
42,YouGov,Economist,5/23/2025,5/26/2025,1660.0,A,44.0,52.0,44.250,51.750,-8.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,North Star Opinion Research,Defending Democracy Together,3/13/2025,3/17/2025,1000.0,RV,46.0,51.0,46.000,51.000,-5.0,https://www.defendingdemocracytogether.org/app...,0.043339,324
98,University of Massachusetts (Amherst),WCVB-TV,4/4/2025,4/9/2025,1000.0,A,44.0,51.0,44.000,51.000,-7.0,https://www.umass.edu/political-science/about/...,0.536200,325
283,OnMessage Inc.,Rescuing the American Dream,2/17/2025,2/19/2025,1000.0,LV,50.0,49.0,48.225,50.775,1.0,https://onmessageinc.com/wp-content/uploads/20...,-0.600249,326
298,SurveyUSA,NaN,2/13/2025,2/16/2025,2000.0,A,51.0,45.0,50.765,45.235,6.0,https://www.surveyusa.com/client/PollReport.as...,-0.732433,327


In [133]:
def avgs_pipeline(data: pd.DataFrame, date: datetime.date):
    state_pivot = pipeline(data)
    # state_pivot['end_date'] = pd.to_datetime(state_pivot['end_date'])
    # state_pivot['pred_plus_minus'] = state_pivot['pred_plus_minus'].map(lambda x: -x)
    state_pivot = state_pivot[state_pivot['end_date'] <= pd.to_datetime(date)]
    state_pivot['population'] = state_pivot['population'].astype(str).map(lambda x: x.strip())
    
    total_num_polls = state_pivot.shape[0]

    # Sample size weights
    total_sample_size = np.sum(state_pivot['sample_size'])
    state_pivot['sample_size_weights'] = (state_pivot['sample_size'].map(lambda x: np.sqrt(min(x, 3000))) / np.sqrt(np.median(state_pivot['sample_size'].map(lambda x: min(x, 2000)))))
    # print(np.median(state_pivot['sample_size'].map(lambda x: min(x, 5000))))
    state_pivot['sample_size_weights'] /= np.sum(state_pivot['sample_size_weights'])
    
    # Time weights
    # Variation of the equation used here: https://polls.votehub.us/
    latest_date = pd.Timestamp(date)
    delta = state_pivot['end_date'].apply(lambda x: (latest_date - x).days)# (latest_date - state_pivot['end_date']).apply(lambda x: x.days)
    linear_weights = (1 - delta/((latest_date - state_pivot['end_date'].min()).days + 1))
    exp_weights = 0.1**(delta/(30))
    state_pivot['time_weights'] =  0 * linear_weights + 1 * exp_weights
    # state_pivot['time_weights'] /= np.sum(state_pivot['time_weights'])
    
    # Quality weights
    rel_quality = state_pivot['pred_plus_minus']
    def quality_weight(rel_qual):
        if rel_qual > 0.5: # Utilizing predictive plus-minus from Silver Bulletin's ratings
            return 0.2
        # return np.sqrt(-rel_qual+1)+0.05
        return np.exp(-rel_qual/1.3)
    state_pivot['quality_weights'] = rel_quality.map(quality_weight)
    # state_pivot['quality_weights'] /= np.sum(state_pivot['quality_weights'])
    
    # Population weights
    def population_weight(population):
        if population == 'RV':
            return 0.8
        elif population == 'LV':
            return 0.6
        return 1
    state_pivot['population_weights'] = state_pivot['population'].map(population_weight)
    # state_pivot['population_weights'] /= np.sum(state_pivot['population_weights'])

    # Gather the weights together
    state_pivot['total_weights'] = state_pivot['sample_size_weights'] * state_pivot['time_weights'] * state_pivot['quality_weights'] * state_pivot['population_weights']
    state_pivot['total_weights'] /= np.sum(state_pivot['total_weights']) # Normalization step
    
    return state_pivot

In [134]:
approval_polls['sample_size'].isna().any()

False

In [135]:
avgs_pipeline(approval_polls, datetime.datetime.today())

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id,sample_size_weights,time_weights,quality_weights,population_weights,total_weights
10,YouGov,Economist,6/13/2025,2025-06-16,1512.0,A,41.0,54.0,41.250,53.750,-13.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,105,0.004763,0.681292,0.964021,1.0,6.076094e-02
21,YouGov,Economist,6/6/2025,2025-06-09,1533.0,A,43.0,52.0,43.250,51.750,-9.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,106,0.004796,0.398107,0.964021,1.0,3.575085e-02
23,YouGov,CBS News,6/4/2025,2025-06-06,2428.0,A,45.0,55.0,45.250,54.750,-10.0,https://www.cbsnews.com/news/deportation-immig...,0.047635,107,0.006036,0.316228,0.964021,1.0,3.573877e-02
30,YouGov,Economist,5/30/2025,2025-06-02,1610.0,A,45.0,49.0,45.250,48.750,-4.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,108,0.004915,0.232631,0.964021,1.0,2.140893e-02
42,YouGov,Economist,5/23/2025,2025-05-26,1660.0,A,44.0,52.0,44.250,51.750,-8.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,109,0.004991,0.135936,0.964021,1.0,1.270289e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,North Star Opinion Research,Defending Democracy Together,3/13/2025,2025-03-17,1000.0,RV,46.0,51.0,46.000,51.000,-5.0,https://www.defendingdemocracytogether.org/app...,0.043339,324,0.003874,0.000631,0.967212,0.8,3.673165e-05
98,University of Massachusetts (Amherst),WCVB-TV,4/4/2025,2025-04-09,1000.0,A,44.0,51.0,44.000,51.000,-7.0,https://www.umass.edu/political-science/about/...,0.536200,325,0.003874,0.003687,0.200000,1.0,5.547860e-05
283,OnMessage Inc.,Rescuing the American Dream,2/17/2025,2025-02-19,1000.0,LV,50.0,49.0,48.225,50.775,1.0,https://onmessageinc.com/wp-content/uploads/20...,-0.600249,326,0.003874,0.000086,1.586817,0.6,6.143845e-06
298,SurveyUSA,NaN,2/13/2025,2025-02-16,2000.0,A,51.0,45.0,50.765,45.235,6.0,https://www.surveyusa.com/client/PollReport.as...,-0.732433,327,0.005478,0.000068,1.756652,1.0,1.273395e-05


In [136]:
def approval_over_time(data: pd.DataFrame):
    date_range = pd.date_range(start=datetime.date(2025, 1, 23), end=datetime.datetime.today(), freq='d', inclusive='both')
    yes_averages = []
    no_averages = []
    yes_stdevs = []
    no_stdevs = []
    for date in date_range:
        pipelined_df = avgs_pipeline(data, date)
        yes_avg = np.sum(pipelined_df['approve_adj'] * pipelined_df['total_weights'])
        no_avg = np.sum(pipelined_df['disapprove_adj'] * pipelined_df['total_weights'])
        yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['approve_adj'] - yes_avg)**2))
        no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['disapprove_adj'] - no_avg)**2))
        yes_averages.append(yes_avg)
        no_averages.append(no_avg)
        yes_stdevs.append(yes_std)
        no_stdevs.append(no_std)
    
    return pd.DataFrame({'Date':date_range, 'Approve':yes_averages, 'Disapprove':no_averages}), pd.DataFrame({'Date':date_range, 'Approve':yes_stdevs, 'Disapprove':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())

In [137]:
approval_avg, approval_std, polls_df = approval_over_time(approval_polls)
approval_avg

,Date,Approve,Disapprove
0,2025-01-23,51.302546,45.716945
1,2025-01-24,51.229190,45.375031
2,2025-01-25,51.229190,45.375031
3,2025-01-26,50.557686,45.329878
4,2025-01-27,49.349467,45.485371
...,...,...,...
145,2025-06-17,45.024266,51.594162
146,2025-06-18,45.160265,51.612710
147,2025-06-19,45.730982,51.202784
148,2025-06-20,45.730982,51.202784


In [138]:
polls_df.shape

(202, 19)

In [139]:
polls_df.head(3)

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id,sample_size_weights,time_weights,quality_weights,population_weights,total_weights
10,YouGov,Economist,6/13/2025,2025-06-16,1512.0,A,41.0,54.0,41.25,53.75,-13.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,105,0.004763,0.681292,0.964021,1.0,0.060761
21,YouGov,Economist,6/6/2025,2025-06-09,1533.0,A,43.0,52.0,43.25,51.75,-9.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,106,0.004796,0.398107,0.964021,1.0,0.035751
23,YouGov,CBS News,6/4/2025,2025-06-06,2428.0,A,45.0,55.0,45.25,54.75,-10.0,https://www.cbsnews.com/news/deportation-immig...,0.047635,107,0.006036,0.316228,0.964021,1.0,0.035739


In [140]:
polls_df['net_adj'] = polls_df['approve_adj'] - polls_df['disapprove_adj']

In [141]:
display_df = polls_df[['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size', 'population', 'approve', 'disapprove', 'net',
                      'total_weights', 'url']]
display_df['sponsors'] = '^Sponsor: ' + display_df['sponsors'] + '^'
display_df['sponsors'] = display_df['sponsors'].fillna('')
display_df = display_df.sort_values(['end_date'], ascending=False)
display_df = display_df.rename({'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 'total_weights':'Weight', 'startdate':'Start Date', 'end_date':'End Date', 'url':'URL'}, axis='columns')
display_df['Sample'] = display_df['sample_size'].astype(int).astype(str) + ' ' + display_df['population']
# display_df['end_date'] = display_df['end_date'].map(lambda x: datetime.datetime.strptime(datetime.datetime.fromtimestamp(x), '%m/%d/%Y').strftime('%Y-%m-%d'))
# display_df['Date'] = pd.to_datetime(display_df['startdate']).astype(str) + ' - ' + display_df['end_date'].astype(str)
display_df['Start Date'] = pd.to_datetime(display_df['Start Date']).astype(str)
display_df['End Date'] = display_df['End Date'].astype(str)
display_df = display_df.reset_index()
# display_df = display_df.drop(['sample_size', 'population'], axis='columns')
display_df['Pollster'] = '<a href="' + display_df['URL'] + '" style="color:black; text-decoration:underline;"><u>' + display_df['Pollster'] + '</u></a>' + display_df['sponsors']
# display_df['Pollster'] = '[' + display_df['Pollster'] + '](' + display_df['URL'] + ')'
display_df = display_df[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net']]
# display_df = display_df.set_index(['Pollster'])
display_df

,Pollster,Start Date,End Date,Sample,Weight,Approve,Disapprove,Net
0,"<a href=""https://napolitannews.org/jobapproval...",2025-06-11,2025-06-19,3000 RV,7.219929e-02,53.0,46.0,7.000000
1,"<a href=""https://echelonin.wpenginepowered.com...",2025-06-17,2025-06-18,1002 LV,4.240315e-02,48.0,52.0,-4.000000
2,"<a href=""https://jlpartners.com/israeliran"" st...",2025-06-16,2025-06-17,1039 RV,3.875936e-02,46.0,51.0,-5.000000
3,"<a href=""https://echelonin.wpenginepowered.com...",2025-06-12,2025-06-17,982 LV,3.887645e-02,45.0,53.0,-8.000000
4,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-06-13,2025-06-16,1512 A,6.076094e-02,41.0,54.0,-13.000000
...,...,...,...,...,...,...,...,...
197,"<a href=""https://researchco.ca/2025/01/27/immi...",2025-01-22,2025-01-24,1001 A,1.755264e-07,50.0,46.0,4.000000
198,"<a href=""https://x.com/ScottWRasmussen/status/...",2025-01-20,2025-01-23,3000 RV,9.089352e-07,57.0,39.0,18.000000
199,"<a href=""https://cdn.atlasintel.org/ed3f5238-e...",2025-01-21,2025-01-23,1882 A,2.328235e-06,50.1,49.7,0.399998
200,"<a href=""https://www.ipsos.com/en-us/reuters-i...",2025-01-20,2025-01-21,1077 A,5.143318e-07,47.0,41.0,6.000000


In [142]:
display_df.to_csv('polls_display_table.csv')

In [143]:
approval_std

,Date,Approve,Disapprove
0,2025-01-23,3.697758,5.575750
1,2025-01-24,3.393680,5.186733
2,2025-01-25,3.393680,5.186733
3,2025-01-26,3.666409,4.754769
4,2025-01-27,3.674859,4.356649
...,...,...,...
145,2025-06-17,3.576593,3.304008
146,2025-06-18,3.548745,3.228737
147,2025-06-19,3.983715,3.439693
148,2025-06-20,3.983715,3.439693


In [144]:
approval_avg['Net'] = approval_avg['Approve'] - approval_avg['Disapprove']
approval_avg

,Date,Approve,Disapprove,Net
0,2025-01-23,51.302546,45.716945,5.585601
1,2025-01-24,51.229190,45.375031,5.854159
2,2025-01-25,51.229190,45.375031,5.854159
3,2025-01-26,50.557686,45.329878,5.227807
4,2025-01-27,49.349467,45.485371,3.864096
...,...,...,...,...
145,2025-06-17,45.024266,51.594162,-6.569896
146,2025-06-18,45.160265,51.612710,-6.452445
147,2025-06-19,45.730982,51.202784,-5.471802
148,2025-06-20,45.730982,51.202784,-5.471802


In [145]:
approval_trendline = pd.merge(left=approval_avg, right=approval_std, left_on='Date', right_on='Date', how='inner')
approval_trendline['approve_CI_upper'] = approval_trendline['Approve_x'] + 1.96*approval_trendline['Approve_y']
approval_trendline['approve_CI_lower'] = approval_trendline['Approve_x'] - 1.96*approval_trendline['Approve_y']
approval_trendline['disapprove_CI_upper'] = approval_trendline['Disapprove_x'] + 1.96*approval_trendline['Disapprove_y']
approval_trendline['disapprove_CI_lower'] = approval_trendline['Disapprove_x'] - 1.96*approval_trendline['Disapprove_y']
approval_trendline = approval_trendline.rename({
    'Approve_x':'approve_mean',
    'Disapprove_x':'disapprove_mean',
}, axis='columns').drop(['Approve_y', 'Disapprove_y', 'Net'], axis='columns')
approval_trendline

,Date,approve_mean,disapprove_mean,approve_CI_upper,approve_CI_lower,disapprove_CI_upper,disapprove_CI_lower
0,2025-01-23,51.302546,45.716945,58.550152,44.054940,56.645416,34.788474
1,2025-01-24,51.229190,45.375031,57.880803,44.577578,55.541028,35.209034
2,2025-01-25,51.229190,45.375031,57.880803,44.577578,55.541028,35.209034
3,2025-01-26,50.557686,45.329878,57.743848,43.371524,54.649225,36.010531
4,2025-01-27,49.349467,45.485371,56.552191,42.146743,54.024404,36.946338
...,...,...,...,...,...,...,...
145,2025-06-17,45.024266,51.594162,52.034389,38.014143,58.070019,45.118306
146,2025-06-18,45.160265,51.612710,52.115806,38.204725,57.941034,45.284386
147,2025-06-19,45.730982,51.202784,53.539063,37.922900,57.944581,44.460986
148,2025-06-20,45.730982,51.202784,53.539063,37.922900,57.944581,44.460986


In [146]:
# Export approval ratings
approval_trendline.to_csv('approval_stats.csv')

In [147]:
polls_df.shape

(202, 20)

In [148]:
line = px.line(approval_avg, x='Date', y=['Approve', 'Disapprove'])
# scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['approve', 'disapprove'])
line.update_traces(hovertemplate=None)
line.update_layout(hovermode='x unified')
# scatter.update_traces(hovertemplate=None, hoverinfo='skip')
fig_approve_CI = go.Figure([
    go.Scatter(
        name='Approve CI Upper Bound',
        x = approval_avg['Date'],
        y = approval_avg['Approve'] + 1.96*approval_std['Approve'],
        mode='lines',
        marker=dict(color='#8be8f2'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    go.Scatter(
        name='Approve CI Lower Bound',
        x = approval_avg['Date'],
        y = approval_avg['Approve'] - 1.96*approval_std['Approve'],
        mode='lines',
        marker=dict(color='#8be8f2'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),  
])
fig_disapprove_CI = go.Figure([
    go.Scatter(
        name='Disapprove CI Upper Bound',
        x = approval_avg['Date'],
        y = approval_avg['Disapprove'] + 1.96*approval_std['Disapprove'],
        mode='lines',
        marker=dict(color='#a3cbec'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    go.Scatter(
        name='Disapprove CI Lower Bound',
        x = approval_avg['Date'],
        y = approval_avg['Disapprove'] - 1.96*approval_std['Disapprove'],
        mode='lines',
        marker=dict(color='#a3cbec'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),  
])

fig = go.Figure(data=line.data + fig_approve_CI.data + fig_disapprove_CI.data)
fig.update_layout(
    title='President Trump Approval Rating',
    xaxis_title='Date',
    yaxis_title='%',
    legend_title='Legend',
    hovermode='x unified'
)
fig.show()

In [149]:
line = px.line(approval_avg, x='Date', y=['Net'])
scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['net'])
                     #trendline='lowess', trendline_options=dict(frac=0.2))
fig = go.Figure(data=line.data + scatter.data)
fig.add_hline(y=0, line_dash='dash')
fig.show()

In [150]:
# scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['net', 'net_adj'])
# fig = go.Figure(data=scatter.data)
# fig.add_hline(y=0, line_dash='dash')
# fig.show()

In [151]:
def net_col(df: pd.DataFrame, min_date: datetime.date):
#     df_copy = df.set_index(['Date'])
#     return df_copy['approve_mean'] - df_copy['disapprove_mean']
    date_range = pd.date_range(start=min_date, end=datetime.datetime.today(), freq='d', inclusive='both')
    data = df.copy()
    data['net_adj'] = data['approve_adj'] - data['disapprove_adj']
    net_averages = []
    net_stdevs = []
    issue_cols = {'Date': date_range, 'net':[], 'net_std':[]}
    for date in date_range:
    #         try:
            pipelined_df = avgs_pipeline(data, date)
    #         except TypeError:
    #             print('TypeError')
    #             print(date)
            net_avg = np.sum(pipelined_df['net_adj'] * pipelined_df['total_weights'])
            net_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['net_adj'] - net_avg)**2))

            net_averages.append(net_avg)
            net_stdevs.append(net_std)

    issue_cols['net'].extend(net_averages)
    issue_cols[f'net_std'].extend(net_stdevs)
        
    net_averages, net_stdevs = [], []
        
    return pd.DataFrame(issue_cols)

In [152]:
net_approval = net_col(approval, datetime.date(2025, 1, 23))
net_approval

,Date,net,net_std
0,2025-01-23,5.585601,8.638245
1,2025-01-24,5.854159,7.957334
2,2025-01-25,5.854159,7.957334
3,2025-01-26,5.227807,7.578410
4,2025-01-27,3.864096,6.999819
...,...,...,...
145,2025-06-17,-6.569896,6.285198
146,2025-06-18,-6.452445,6.163305
147,2025-06-19,-5.471802,6.899392
148,2025-06-20,-5.471802,6.899392


In [153]:
net_approval['net_upper_CI'] = net_approval['net'] + 1.96*net_approval['net_std']
net_approval['net_lower_CI'] = net_approval['net'] - 1.96*net_approval['net_std']
net_approval

,Date,net,net_std,net_upper_CI,net_lower_CI
0,2025-01-23,5.585601,8.638245,22.516561,-11.345359
1,2025-01-24,5.854159,7.957334,21.450534,-9.742216
2,2025-01-25,5.854159,7.957334,21.450534,-9.742216
3,2025-01-26,5.227807,7.578410,20.081490,-9.625875
4,2025-01-27,3.864096,6.999819,17.583741,-9.855549
...,...,...,...,...,...
145,2025-06-17,-6.569896,6.285198,5.749092,-18.888885
146,2025-06-18,-6.452445,6.163305,5.627633,-18.532522
147,2025-06-19,-5.471802,6.899392,8.051006,-18.994610
148,2025-06-20,-5.471802,6.899392,8.051006,-18.994610


In [154]:
net_approval.to_csv('net_approval.csv')

### Auxiliary Graph: Individual Pollsters

In [155]:
approval_polls.columns.values

array(['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size',
       'population', 'approve', 'disapprove', 'approve_adj',
       'disapprove_adj', 'net', 'url', 'pred_plus_minus', 'poll_id'],
      dtype=object)

In [156]:
appr = full_approval.copy()
appr['net_adj'] = appr['approve'] - appr['disapprove']
pollsters = appr['pollster'].value_counts().index.values
pollsters

array(['Rasmussen Reports', 'YouGov', 'Morning Consult', 'RMG Research',
       'Ipsos', 'J.L. Partners', 'Echelon Insights', 'TIPP Insights',
       'Harris Insights & Analytics',
       'Global Strategy Group/GBAO (Navigator Research)',
       'Quinnipiac University', 'Emerson College', 'Gallup', 'AtlasIntel',
       'Clarity Campaign Labs', 'SSRS', 'McLaughlin & Associates',
       'American Research Group', 'Pew Research Center',
       'InsiderAdvantage', 'John Zogby Strategies', 'Civiqs',
       'Beacon Research/Shaw & Co. Research', 'co/efficient', 'Cygnal',
       'Marquette University Law School', 'Marist College',
       'Blueprint Polling', 'Fabrizio Lee & Associates/Impact Research',
       'Hart Research Associates/Public Opinion Strategies',
       'SurveyMonkey', 'Public Opinion Strategies', 'SurveyUSA',
       'OnMessage Inc.', 'University of Massachusetts (Amherst)',
       'North Star Opinion Research', 'Remington Research Group',
       'OH Predictive Insights', 'Pub

In [157]:
def approval_today(data: pd.DataFrame):
    # date_range = pd.date_range(start=datetime.date(2025, 1, 23), end=datetime.datetime.today(), freq='d', inclusive='both')
#     yes_averages = []
#     no_averages = []
#     yes_stdevs = []
#     no_stdevs = []
    pipelined_df = avgs_pipeline(data, datetime.datetime.today())
    yes_avg = np.sum(pipelined_df['approve'] * pipelined_df['total_weights'])
    no_avg = np.sum(pipelined_df['disapprove'] * pipelined_df['total_weights'])
    yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['approve'] - yes_avg)**2))
    no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['disapprove'] - no_avg)**2))
    net_avg = np.sum(pipelined_df['net'] * pipelined_df['total_weights'])
    net_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['net'] - net_avg)**2))
    
    #return pd.DataFrame({'Date':date_range, 'Approve':yes_averages, 'Disapprove':no_averages}), pd.DataFrame({'Date':date_range, 'Approve':yes_stdevs, 'Disapprove':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())
    return yes_avg, no_avg, yes_std, no_std, net_avg, net_std

In [158]:
def indiv_pollster_avg(pollster: str):
    df = appr[appr['pollster'] == pollster]
    yes_avg, no_avg, yes_std, no_std, net_avg, net_std = approval_today(df)
    return yes_avg, no_avg, yes_std, no_std, net_avg, net_std

In [159]:
def get_ppm(pollster: str):
    df = ratings[ratings['Pollster'] == pollster]
    ppm = df['Predictive Plus-Minus'].values[0]
    
    return ppm

In [160]:
appr.shape

(334, 26)

In [161]:
def get_num_polls(pollster: str):
    df = appr[appr['pollster'] == pollster]
    num_polls = df.shape[0]
    return num_polls

In [162]:
get_ppm('YouGov')

0.047635015

In [163]:
indiv_pollster_avg('YouGov')

(42.900474191057825,
 52.76860451019764,
 1.669632413956023,
 1.9805638285980822,
 -9.86813031913983,
 2.950622566756215)

In [164]:
indiv_pollster_avg('RMG Research')

(51.74562957131361,
 46.68340935677874,
 1.533554624310377,
 1.4319488836630792,
 5.062220214534894,
 2.866232675147639)

In [165]:
nets = []
net_stdevs = []
ppms = []
nums = []

for p in pollsters:
    _, _, _, _, net, net_std = indiv_pollster_avg(p)
    ppm = get_ppm(p)
    num_polls = get_num_polls(p)
    
    nets.append(net)
    net_stdevs.append(net_std)
    ppms.append(ppm)
    nums.append(num_polls)

In [166]:
# nets

In [167]:
indiv_avgs = pd.DataFrame({'Pollster': pollsters, 'Net Approval': nets, 'Net Stdev': net_stdevs, 'Predictive Plus-Minus': ppms, 'Number of Approval Polls Conducted': nums})
indiv_avgs = indiv_avgs.sort_values(['Net Approval'], ascending=True)
indiv_avgs

,Pollster,Net Approval,Net Stdev,Predictive Plus-Minus,Number of Approval Polls Conducted
15,SSRS,-17.494832,2.384042,0.966835,4
18,Pew Research Center,-17.026012,0.251962,0.132230,3
10,Quinnipiac University,-15.959728,0.434969,-0.290484,5
17,American Research Group,-13.567189,1.349999,0.818141,4
9,Global Strategy Group/GBAO (Navigator Research),-12.410504,1.263552,0.804988,7
42,NewsNation,-12.000000,0.000000,0.614459,1
43,The New York Times/Siena College,-12.000000,0.000000,-0.763947,1
4,Ipsos,-11.743412,1.023970,0.442898,15
38,Public Religion Research Institute,-11.000000,0.000000,0.103864,1
26,Marist College,-10.898258,0.837759,-0.436610,2


In [168]:
indiv_avgs.to_csv('indiv_pollster_avgs.csv')

In [169]:
reg = lm.LinearRegression()
reg.fit(indiv_avgs[['Predictive Plus-Minus']], indiv_avgs['Net Approval'])
reg.coef_, reg.intercept_

(array([-1.39598659]), -5.0702722790277885)

In [170]:
r = r_regression(indiv_avgs[['Predictive Plus-Minus']], indiv_avgs['Net Approval'])
r

array([-0.11917975])